<center>
# МЕТОДИЧЕСКАЯ ИНСТРУКЦИЯ ПО РЕШЕНИЮ ЗАДАЧИ ПРЕДСКАЗАНИЯ ОТКЛИКА ОТП БАНКА

In [1]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Verdana')

In [2]:
# импортируем класс RobustScaler
from sklearn.preprocessing import RobustScaler
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score
from sklearn.metrics import roc_auc_score

# Считывание CSV-файла, содержащего исторические данные, в объект DataFrame

In [3]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор данных
data = pd.read_csv("C:/Trees/Credit_OTP.csv", encoding='cp1251', sep = ';')
# выводим первые 5 наблюдений
data.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.0,0,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.0,1,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.0,0,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.0,1,1,1,1,1,6,3,1,"1589,923333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.0,1,0,1,2,1,16,2,1,"1152,15",2230,NaN


In [4]:
# выводим информацию о количестве непропущенных 
# наблюдений в переменных и типах переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null int64
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null int64
SOCSTATUS_PENS_FL             15223 non-null int64
GENDER                        15223 non-null int64
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null object
REG_ADDRESS_PRO

# Предварительная обработка исторических данных

In [5]:
def preprocessing(df):
    # удаляем идентификационную переменную AGREEMENT_RK,
    # потому что у нее количество уникальных значений
    # равно количеству наблюдений
    df.drop('AGREEMENT_RK', axis=1, inplace=True)
    
    # удаляем переменную DL_DOCUMENT_FL, потому что 
    # у нее одно уникальное значение
    df.drop(['DL_DOCUMENT_FL'], axis=1, inplace=True)
    
    # преобразуем указанные переменные в тип object
    for i in ['TARGET', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'GENDER', 
              'EDUCATION', 'MARITAL_STATUS', 'GEN_INDUSTRY', 'GEN_TITLE',
              'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'REG_ADDRESS_PROVINCE',
              'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE', 'REGION_NM',
              'REG_FACT_FL', 'FACT_POST_FL', 'REG_POST_FL', 'REG_FACT_POST_FL', 
              'REG_FACT_POST_TP_FL', 'FL_PRESENCE_FL', 'AUTO_RUS_FL', 'HS_PRESENCE_FL',
              'COT_PRESENCE_FL', 'GAR_PRESENCE_FL', 'LAND_PRESENCE_FL', 'GPF_DOCUMENT_FL', 
              'FACT_PHONE_FL', 'REG_PHONE_FL', 'GEN_PHONE_FL']:
        if i in df.columns:
            df[i]=df[i].astype('object')
    
    # в указанных переменных заменяем запятую на точку в качестве
    # десятичного разделителя и преобразуем в тип float
    for i in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
        if i in df.columns:
            df[i] = df[i].str.replace(',', '.').astype('float')
    
    
    # если в интересующей нас переменной есть пропуск 
    # и при этом переменная SOCSTATUS_PENS_FL имеет значение 1,
    # заменяем такие пропуски меткой "Не указано"
    df['GEN_INDUSTRY'] = np.where(df['GEN_INDUSTRY'].isnull() | (df['SOCSTATUS_PENS_FL']==1), 
                                  'Не указано', df['GEN_INDUSTRY'])
    df['GEN_TITLE'] = np.where(df['GEN_TITLE'].isnull() | (df['SOCSTATUS_PENS_FL']==1), 
                               'Не указано', df['GEN_TITLE'])
    df['ORG_TP_STATE'] = np.where(df['ORG_TP_STATE'].isnull() | (df['SOCSTATUS_PENS_FL']==1),
                                  'Не указано', df['ORG_TP_STATE'])
    df['ORG_TP_FCAPITAL'] = np.where(df['ORG_TP_FCAPITAL'].isnull() | (df['SOCSTATUS_PENS_FL']==1), 
                                     'Не указано', df['ORG_TP_FCAPITAL'])

    # заменяем пропуски в указанных переменных
    # меткой "Не указано"
    df['JOB_DIR']=np.where(df['JOB_DIR'].isnull(), 'Не указано', df['JOB_DIR'])
    df['TP_PROVINCE']=np.where(df['TP_PROVINCE'].isnull(), 'Не указано', df['TP_PROVINCE'])
    df['REGION_NM']=np.where(df['REGION_NM'].isnull(), 'Не указано', df['REGION_NM'])
    
    # заменяем пропуски в переменной 
    # PREVIOUS_CARD_NUM_UTILIZED нулями
    df['PREVIOUS_CARD_NUM_UTILIZED']=np.where(df['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                              df['PREVIOUS_CARD_NUM_UTILIZED'])
    
    # заменяем категорию "Не указано" на категорию "ЮЖНЫЙ"
    df['REGION_NM']=np.where(df['REGION_NM']=='Не указано', 'ЮЖНЫЙ', df['REGION_NM'])
    
    # заменяем неверную категорию "ПОВОЛЖСКИЙ" на категорию "ПРИВОЛЖСКИЙ"
    df.at[df['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'
    
    # записываем редкие категории в одну отдельную категорию
    for i in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
        if i in df.columns:
            df[i]=np.where((df[i]=='Москва') \
                           | (df[i]=='Хакасия') \
                           | (df[i]=='Ямало-Ненецкий АО') \
                           | (df[i]=='Магаданская область') \
                           | (df[i]=='Калмыкия') \
                           | (df[i]=='Дагестан') \
                           | (df[i]=='Агинский Бурятский АО') \
                           | (df[i]=='Усть-Ордынский Бурятский АО') \
                           | (df[i]=='Эвенкийский АО') \
                           | (df[i]=='Коми-Пермяцкий АО') \
                           | (df[i]=='Чечня'), 
                           'ДРУГОЕ', df['REG_ADDRESS_PROVINCE'])


    df['TP_PROVINCE']=np.where((df['TP_PROVINCE']=='Сахалинская область') \
                               | (df['TP_PROVINCE']=='Еврейская АО') \
                               | (df['TP_PROVINCE']=='Магаданская область') \
                               | (df['TP_PROVINCE']=='Москва') \
                               | (df['TP_PROVINCE']=='Кабардино-Балкария'), 
                               'ДРУГОЕ', df['TP_PROVINCE'])
    
    # укрупняем категории переменной EDUCATION
    df.at[df['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Высшее'
    df.at[df['EDUCATION'] == 'Два и более высших образования', 'EDUCATION'] = 'Высшее'

    # записываем некоторые категории переменной GEN_INDUSTRY 
    # в отдельную категорию
    df.at[df['GEN_INDUSTRY'] == 'Юридические услуги/нотариальные услуги', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Страхование', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Туризм', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Недвижимость', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Управляющая компания', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Логистика', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Подбор персонала', 'GEN_INDUSTRY'] = 'Другие сферы'
    df.at[df['GEN_INDUSTRY'] == 'Маркетинг', 'GEN_INDUSTRY'] = 'Другие сферы'
    
    # укрупняем категории переменной GEN_TITLE
    df.at[df['GEN_TITLE'] == 'Партнер', 'GEN_TITLE'] = 'Другое'
    df.at[df['GEN_TITLE'] == 'Военнослужащий по контракту', 'GEN_TITLE'] = 'Другое'
    
    # укрупняем категории переменной ORG_TP_STATE
    df.at[df['ORG_TP_STATE'] == 'Частная ком. с инос. капиталом', 'ORG_TP_STATE'] = 'Частная компания'
    
    # укрупняем категории переменной JOB_DIR
    df.at[df['JOB_DIR'] == 'Реклама и маркетинг', 'JOB_DIR'] = 'Другое'
    df.at[df['JOB_DIR'] == 'Кадровая служба и секретариат', 'JOB_DIR'] = 'Другое'
    df.at[df['JOB_DIR'] == 'Пр-техн. обесп. и телеком.', 'JOB_DIR'] = 'Другое'
    df.at[df['JOB_DIR'] == 'Юридическая служба', 'JOB_DIR'] = 'Другое'
    
    # создаем переменную FACT_TP_FL, которая принимает значение 1, если 
    # область фактического пребывания клиента и область торговой точки, 
    # где клиент брал последний кредит, совпадают, или 0
    # в противном случае
    df['FACT_TP_FL'] = np.where(df['FACT_ADDRESS_PROVINCE']==df['TP_PROVINCE'], 1, 0).astype('object')
    
    # создаем переменную AUTO_FOR_FL, которая принимает значение 1, если 
    # у клиента есть один или несколько иностранных автомобилей, или
    # 0 в противном случае
    df['AUTO_FOR_FL'] = np.where((df['AUTO_RUS_FL']=='0') | (df['OWN_AUTO']>0), 1, 0).astype('object')
    
    # создаем переменную - результат конъюнкции GENDER и FAMILY_INCOME
    df["GENDER+FAMILY_INCOME"]= df["GENDER"].astype(str) + ' + ' + df["FAMILY_INCOME"].astype(str)
    
    # значения переменной FACT_LIVING_TERM берем по модулю, чтоб избавиться от отрицательных значений
    df['FACT_LIVING_TERM']=df['FACT_LIVING_TERM'].abs()
    
    # наблюдения, в которых количество лет проживания 
    # по месту фактического пребывания, превышает
    # возраст, записываем как пропуски 
    df['FACT_LIVING_TERM'] = np.where(df['FACT_LIVING_TERM']/12>df['AGE'], np.NaN, df['FACT_LIVING_TERM'])
    
    # импутируем пропуски медианой
    df['FACT_LIVING_TERM'].fillna(df['FACT_LIVING_TERM'].median(), inplace=True)
    
    # наблюдения, в которых время работы в годах превышает 
    # возраст (например, человек работает 40 лет, а живет 
    # всего 25), записываем как пропуски 
    df['WORK_TIME'] = np.where(df['WORK_TIME']/12>df['AGE'], np.NaN, df['WORK_TIME'])
    
    # наблюдения, в которых разница между возрастом и временем работы в годах 
    # меньше 16 (например, у 30-летнего время работы в годах составляет 20 лет,
    # получается, он работает с 10 лет), записываем как пропуски 
    df['WORK_TIME'] = np.where((df['AGE']-df['WORK_TIME']/12)<16, np.NaN, df['WORK_TIME'])
    
    # импутируем пропуски в переменной WORK_TIME медианой
    df['WORK_TIME'].fillna(df['WORK_TIME'].median(), inplace=True)
    
    # на основе переменной PERSONAL_INCOME создаем переменную 
    # PERSONAL_INCOME_INT, у которой метки категории будут 
    # повторять метки категорий FAMILY_INCOME
    bins = [-np.inf, 5000, 10000, 20000, 50000, np.inf]
    lab = ['до 5000 руб.', 'от 5000 до 10000 руб.', 'от 10000 до 20000 руб.', 
           'от 20000 до 50000 руб.', 'свыше 50000 руб.']
    df['PERSONAL_INCOME_INT']=pd.cut(df['PERSONAL_INCOME'], bins, labels=lab)
    
    # создаем переменную PERSONAL_FAMILY_INCOME, которая принимает значение 1, если 
    # категория переменной PERSONAL_FAMILY_INCOME совпадает с категорией 
    # переменной FAMILY_INCOME, или 0 в противном случае, затем
    # преобразовываем в тип object
    df['PERSONAL_FAMILY_INCOME'] = np.where(df['PERSONAL_INCOME_INT']==df['FAMILY_INCOME'], 1, 0).astype('object')
    
    # удаляем переменную PERSONAL_INCOME_INT
    df.drop('PERSONAL_INCOME_INT', axis=1, inplace=True)
    
    # задаем точки, в которых будут находится границы категорий 
    # будущей переменной CREDITCAT
    bins = [-np.inf, 7292, 9427, 14169, 27449, np.inf]
    
    # осуществляем биннинг переменной CREDIT и записываем
    # результаты в новую переменную CREDITCAT
    df['CREDITCAT'] = pd.cut(df['CREDIT'], bins)
    
    # задаем точки, в которых будут находится границы категорий 
    # будущей переменной FSTPAYMENTCAT
    bins = [-np.inf, 1500, 4995, np.inf]
    
    # осуществляем биннинг переменной FST_PAYMENT и записываем
    # результаты в новую переменную FSTPAYMENTCAT
    df['FSTPAYMENTCAT'] = pd.cut(df['FST_PAYMENT'], bins)
    
    # задаем точки, в которых будут находится границы категорий 
    # будущей переменной AGECAT
    bins = [-np.inf, 29, 43, 52, 57, np.inf]
    
    # осуществляем биннинг переменной AGE и записываем
    # результаты в новую переменную AGECAT
    df['AGECAT'] = pd.cut(df['AGE'], bins)
    
    # укрупняем категории переменной GEN_TITLE
    for i in df.index:
        if df.GEN_TITLE[i] in ['Рабочий','Служащий','Работник сферы услуг',                             
                               'Другое', 'Индивидуальный предприниматель',
                               'Руководитель низшего звена']:
            df.set_value(i,'GEN_TITLE',1)
        elif df.GEN_TITLE[i] in ['Специалист','Руководитель среднего звена']: 
            df.set_value(i,'GEN_TITLE',2)
        elif df.GEN_TITLE[i] == 'Не указано':
            df.set_value(i,'GEN_TITLE',3)
        elif df.GEN_TITLE[i] in ['Высококвалифиц. специалист', 'Руководитель высшего звена']:
            df.set_value(i,'GEN_TITLE',4)
    
    # задаем для переменной GEN_TITLE тип object 
    df['GEN_TITLE']=df['GEN_TITLE'].astype('object')
    
    # на основе укрупнения категорий переменной REGION_NM 
    # создадим переменную REGIONCAT
    for i in df.index:
        if df.REGION_NM[i] in ['ЮЖНЫЙ','ЦЕНТРАЛЬНЫЙ 1','ЦЕНТРАЛЬНЫЙ 2']:
            df.set_value(i,'REGIONCAT',1)
        elif df.REGION_NM[i] in ['ЗАПАДНО-СИБИРСКИЙ','ВОСТОЧНО-СИБИРСКИЙ','УРАЛЬСКИЙ',
                                 'ДАЛЬНЕВОСТОЧНЫЙ','СЕВЕРО-ЗАПАДНЫЙ', 'ЦЕНТРАЛЬНЫЙ ОФИС']:
            df.set_value(i,'REGIONCAT',2)
        elif df.REGION_NM[i] == 'ПРИВОЛЖСКИЙ':
            df.set_value(i,'REGIONCAT',3)
        
    # задаем для переменной REGIONCAT тип object 
    df['REGIONCAT']=df['REGIONCAT'].astype('object')
        
    # на основе укрупнения категорий переменной ORG_TP_STATE создаем переменную ORGCAT
    for i in df.index:
        if df.ORG_TP_STATE[i] in ['Частная компания','Индивидуальный предприниматель','Некоммерческая организация']:
            df.set_value(i,'ORGCAT',1)
        elif df.ORG_TP_STATE[i] in ['Не указано']:
            df.set_value(i,'ORGCAT',2)
        elif df.ORG_TP_STATE[i] == 'Государственная комп./учреж.':
            df.set_value(i,'ORGCAT',3)
            
    # задаем для переменной ORGCAT тип object 
    df['ORGCAT']=df['ORGCAT'].astype('object')
    
    # выполняем логарифмическое преобразование
    # переменной PERSONAL_INCOME
    df['PERSONAL_INCOME']=np.log(df['PERSONAL_INCOME']+0.01)
    
    # выполняем логарифмическое преобразование
    # переменной CREDIT
    df['CREDIT']=np.log(df['CREDIT']+0.01)
    
    # выполняем логарифмическое преобразование
    # переменной FACT_LIVING_TERM
    df['WORK_TIME']=np.log(df['WORK_TIME']+0.01)
    
    # выполняем преобразование переменной WORK_TIME
    # кубическим корнем
    df['FACT_LIVING_TERM']=np.sign(df['FACT_LIVING_TERM']) * (df['FACT_LIVING_TERM'].abs()**(1/3))
    
    # выполняем логарифмическое преобразование 
    # переменной LOAN_AVG_DLQ_AMT
    df['LOAN_AVG_DLQ_AMT']=np.log(df['LOAN_AVG_DLQ_AMT']+0.01)
    
    # создаем переменную CHILD_DEPENDANTS
    df['CHILD_DEPENDANTS'] = df['CHILD_TOTAL']/(df['CHILD_TOTAL']+df['DEPENDANTS'])
    
    # если переменная содержит пропуск, то возвращается значение 0, 
    # если пропуска нет, то возвращается исходное значение переменной
    df['CHILD_DEPENDANTS']=np.where(df['CHILD_DEPENDANTS'].isnull(), 0, df['CHILD_DEPENDANTS'])
    
    # создаем переменную PAYMENT - сумму ежемесячного взноса
    # по кредиту, разделив сумму кредита (CREDIT)
    # на срок кредита (TERM)
    df['PAYMENT']=np.log((df['CREDIT']/df['TERM'])+0.01)
    
    # создаем переменную PTI - коэффициент долговой нагрузки, 
    # разделив сумму ежемесячного взноса по кредиту (PAYMENT)
    # на личный доход (PERSONAL_INCOME)
    df['PTI'] = df['PAYMENT']/df['PERSONAL_INCOME']
    
    # создаем переменную CLOSED_TO_TOTAL - коэффициент погашения ссуд, 
    # разделив количество погашенных ссуд (LOAN_NUM_CLOSED)
    # на общее количество ссуд (LOAN_NUM_TOTAL)
    df['CLOSED_TO_TOTAL'] = df['LOAN_NUM_CLOSED']/df['LOAN_NUM_TOTAL']
    
    # создаем переменную PAYM_TO_LOAN– отношение количества 
    # платежей (LOAN_NUM_PAYM) к общему количеству ссуд (LOAN_NUM_TOTAL)
    df['PAYM_TO_LOAN'] = df['LOAN_NUM_PAYM']/df['LOAN_NUM_TOTAL']
    
    # создаем переменную DLQ_TO_PAYM – отношение количества просрочек 
    # (LOAN_DLQ_NUM) к общему количеству платежей (LOAN_NUM_PAYM)
    df['DLQ_TO_PAYM'] = df['LOAN_DLQ_NUM']/df['LOAN_NUM_PAYM']
    
    df['FST_SHARE'] = df['FST_PAYMENT']/(df['FST_PAYMENT']+df['CREDIT'])
    df['DLQ_TIME'] = df['LOAN_MAX_DLQ']/df['LOAN_DLQ_NUM']
    df['DLQ_TIME'] = np.where(df['DLQ_TIME'].isnull(), 0, df['DLQ_TIME'])
    df['LOAN_MAX_DLQ_RANGE']=((df['LOAN_MAX_DLQ_AMT']-df['LOAN_AVG_DLQ_AMT'])/df['LOAN_AVG_DLQ_AMT']).abs()
    df['LOAN_MAX_DLQ_RANGE'] = np.where(df['LOAN_MAX_DLQ_RANGE'].isnull(), 0, df['LOAN_MAX_DLQ_RANGE'])

In [6]:
# выполняем предварительную обработку
# обучающих данных
preprocessing(data)

In [7]:
# выполняем дамми-кодирование
data_dummies = pd.get_dummies(data) 
# создаем обучающий массив меток
y = data_dummies.loc[:, 'TARGET_1']    
# удаляем из будущих массивов признаков результаты
# дамми-кодирования зависимой переменной -
# TARGET_0 и TARGET_1
data_dummies.drop(['TARGET_0', 'TARGET_1'], axis=1, inplace=True)    
# создаем обучающий массив признаков
X = data_dummies.loc[:, 'AGE':'ORGCAT_3.0']
# создаем экземпляр класса RobustScaler
scaler = RobustScaler()
# подгоняем модель
scaler.fit(X)
# преобразовываем данные и по отмасштабированным данным
# уже можно будет строить логистическую регрессию
X = scaler.transform(X) 

# Обучение модели логистической регрессии на всех исторических данных

In [8]:
# строим логистическую регрессию на данных,
# отмасштабированных с помощью RobustScaler
logreg = LogisticRegression(C=0.22, penalty='l1').fit(X, y)
print("AUC на всей исторической выборке: {:.3f}".
      format(roc_auc_score(y, logreg.predict_proba(X)[:, 1])))

AUC на всей исторической выборке: 0.727


# Считывание CSV-файла, содержащего новые данные, в объект DataFrame

In [9]:
# загружаем новые данные
newdata = pd.read_csv("C:/Trees/Credit_OTP_new.csv", encoding='cp1251', sep = ';')

# Предварительная обработка новых данных

In [10]:
# выполняем предварительную обработку
# новых данных
preprocessing(newdata)

In [11]:
# выполняем дамми-кодирование
# новых данных
newdata_dummies = pd.get_dummies(newdata)
# создаем массив меток
y_new = newdata_dummies.loc[:, 'TARGET_1']    
# удаляем из будущих массивов признаков результаты
# дамми-кодирования зависимой переменной -
# TARGET_0 и TARGET_1
newdata_dummies.drop(['TARGET_0', 'TARGET_1'], axis=1, inplace=True)    
# создаем массив признаков
X_new = newdata_dummies.loc[:, 'AGE':'ORGCAT_3.0']
# подгоняем модель
scaler.fit(X_new)
# преобразовываем данные и по отмасштабированным данным
# уже можно будет строить логистическую регрессию
X_new = scaler.transform(X_new) 

# Применение модели логистической регрессии, построенной на всех исторических данных, к новым данным

In [12]:
print("AUC на выборке новых данных: {:.3f}".
      format(roc_auc_score(y_new, logreg.predict_proba(X_new)[:, 1])))

AUC на выборке новых данных: 0.709


# Подключение библиотеки H2O и подготовка данных

In [13]:
# перед импортом библиотеки h2o и модуля os убедитесь, что библиотека h2o установлена 
# (сначала установите Java SE Development Kit 8, обратите внимание, 
# 9-я версия H2O не поддерживается, а затем после установки Java 
# запустите Anaconda Prompt и установите h2o с помощью 
# строки pip install h2o)
import h2o
import os
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,47 mins 42 secs
H2O cluster version:,3.17.0.4124
H2O cluster version age:,3 months and 4 days
H2O cluster name:,H2O_from_python_Gewissta_t6k9mt
H2O cluster total nodes:,1
H2O cluster free memory:,6.939 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [14]:
def cat_to_numbers(df):
    # библиотека h2o пока не поддерживает кириллицу и чтобы не выводилась абракадабра,
    # представим значения категориальных переменных (категории) в виде чисел,
    # по-прежнему обрабатывая их как категориальные
    for i in ['EDUCATION', 'MARITAL_STATUS', 'GEN_INDUSTRY', 'GEN_TITLE', 
              'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'FAMILY_INCOME',
              'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 
              'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE', 'REGION_NM', 
              'GENDER + FAMILY_INCOME']:
        if i in data.columns:
            df[i] = pd.Categorical(df[i])
            df[i] = df[i].cat.codes

In [15]:
def h2o_preprocessing(df):
    # преобразовываем категориальные переменные, неправильно прочитанные 
    # как целочисленные (тип int), в категориальные (тип enum)
    for i in ['TARGET', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'GENDER', 
              'EDUCATION', 'MARITAL_STATUS', 'GEN_INDUSTRY', 'GEN_TITLE',
              'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR', 'FAMILY_INCOME', 
              'REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
              'TP_PROVINCE', 'REGION_NM', 'REG_FACT_FL', 'FACT_POST_FL', 'REG_POST_FL', 
              'REG_FACT_POST_FL', 'REG_FACT_POST_TP_FL', 'FL_PRESENCE_FL', 'AUTO_RUS_FL', 
              'HS_PRESENCE_FL', 'COT_PRESENCE_FL', 'GAR_PRESENCE_FL', 'LAND_PRESENCE_FL',
              'GPF_DOCUMENT_FL', 'FACT_PHONE_FL', 'REG_PHONE_FL', 'GEN_PHONE_FL', 
              'FACT_TP_FL', 'AUTO_FOR_FL', 'GENDER + FAMILY_INCOME', 'PERSONAL_FAMILY_INCOME', 
              'CREDITCAT', 'FSTPAYMENTCAT', 'AGECAT', 'REGIONCAT', 'ORGCAT']:
        if i in df.columns:
            df[i]=df[i].asfactor()

In [16]:
# представляем значения категориальных переменных (категории) в виде чисел,
# по-прежнему обрабатывая их как категориальные
cat_to_numbers(data)
cat_to_numbers(newdata)

In [17]:
# преобразовываем датафреймы pandas во фреймы h2o -
# специальную структуру данных, используемую h2o
frame = h2o.H2OFrame(data)
newframe = h2o.H2OFrame(newdata)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [18]:
# выполняем предварительную 
# подготовку данных
h2o_preprocessing(frame)
h2o_preprocessing(newframe)

In [19]:
# смотрим обучающий фрейм
frame.describe()

Rows:15223
Cols:68




,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,FACT_TP_FL,AUTO_FOR_FL,GENDER+FAMILY_INCOME,PERSONAL_FAMILY_INCOME,CREDITCAT,FSTPAYMENTCAT,AGECAT,REGIONCAT,ORGCAT,CHILD_DEPENDANTS,PAYMENT,PTI,CLOSED_TO_TOTAL,PAYM_TO_LOAN,DLQ_TO_PAYM,FST_SHARE,DLQ_TIME,LOAN_MAX_DLQ_RANGE
type,enum,int,enum,enum,enum,int,int,enum,enum,enum,enum,enum,enum,enum,enum,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,enum,enum,enum,enum,enum,real,int,real,enum,real,real,enum,enum,enum,int,int,int,int,int,real,real,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,real,real,real,real,real,real,real
mins,,21.0,,,,0.0,0.0,,,,,,,,,3.178470410233162,,,,,,,,,,,,0.0,,,,,,7.600907459529583,3.0,0.0,,0.0,0.009950330853168092,,,,1.0,0.0,1.0,0.0,0.0,-4.605170185988091,0.0,0.0,,,,,,,,,,0.0,-1.3111615004499126,-0.14827018163141858,0.0,0.5,0.0,0.0,0.0,1.0
mean,,40.40609603888846,,,,1.0993890823096581,0.6452079090849366,,,,,,,,,9.394342636455086,,,,,,,,,,,,0.11633712146094725,,,,,,9.331847109896612,8.101031334165429,3398.562655192807,,4.61643320066656,3.839946545253821,,,,1.387768508178413,0.7518885896341041,7.875977139854165,0.19536228075937678,0.13019772712343144,-3.09986156509735,243.0216264862374,0.019050121526637324,,,,,,,,,,0.45383823880834967,0.2633829833642283,0.028019445266187784,0.440000015640494,5.14722258242752,0.024652575750191228,0.9149676200461642,0.1069408525084149,32.28470042269361
maxs,,67.0,,,,10.0,7.0,,,,,,,,,12.429216236844384,,,,,,,,,,,,2.0,,,,,,11.69274397508825,36.0,140000.0,,8.962809493114328,6.269115222920853,,,,11.0,11.0,114.0,13.0,8.0,10.240395765534398,28012.2,2.0,,,,,,,,,,1.0,1.2951050395629988,0.13774118377881628,1.0,40.5,0.8333333333333334,0.9999263700432414,1.0,2734.46068349031
sigma,,11.601068034819944,,,,0.9954113720782439,0.8122521216931945,,,,,,,,,0.5181266232146053,,,,,,,,,,,,0.32084410229493804,,,,,,0.7205671564892427,4.094089604434382,5158.108934236151,,1.5499076003280032,1.0964928101055627,,,,0.7935657399633619,0.9884992056042468,7.9557467598192995,0.6850215602247691,0.3557021019613592,3.968842198122912,836.1001158351092,0.13766342869621948,,,,,,,,,,0.3696862768292838,0.44716795859110386,0.047623674365674155,0.4694874431210326,2.445226236868795,0.07974325968166585,0.2690120811294386,0.29696278410734395,100.69204139027566
zeros,,0,,,,4984,8226,,,,,,,,,0,,,,,,,,,,,,13453,,,,,,0,0,1208,,20,0,,,,0,7791,0,13301,13301,0,13301,14935,,,,,,,,,,4984,0,0,7791,0,13301,1208,13301,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,49.0,1,0,1,2.0,1.0,4,4,19,0,4,0,2,1,8.517195191414238,42,42,42,38,3,1,1,1,1,1,0,0.0,0,0,0,0,0,8.987198070661192,6.0,8650.0,1,6.036810736797685,2.8909271591878647,0,0,1,1.0,1.0,6.0,2.0,1.0,7.365186455114908,1580.0,0.0,1,0,1 + <0xEEF2> 10000 <0xE4EE> 20000 <0xF0F3E1>.,0,"(7292.0, 9427.0]","(4995.0, inf]","(43.0, 52.0]",3,1,0.6666666666666666,0.41069563509431506,0.048219587066446114,1.0,6.0,0.3333333333333333,0.9989620959281862,0.5,213.52274285639243
1,0,32.0,1,0,1,3.0,3.0,3,4,19,0,1,0,7,1,9.392662762103123,18,18,18,51,9,1,1,1,1,0,0,0.0,0,0,0,0,0,9.982761195324294,6.0,4000.0,1,5.155136735475772,4.574814065973192,1,0,1,1.0,1.0,6.0,1.0,1.0,8.299039669172162,4020.0,0.0,0,0,1 + <0xEEF2> 10000 <0xE4EE> 20000 <0xF0F3E1>.,1,"(14169.0, 27449.0]","(1500.0, 4995.0]","(29

In [20]:
# задаем название зависимой переменной
dependent = 'TARGET'
# задаем список названий предикторов
predictors = list(frame.columns)
# удаляем название зависимой переменной из 
# списка названий предикторов
predictors.remove(dependent)


# Построение модели логистической регрессии на всех исторических данных и применение к новым данным в H2O

In [21]:
# импортируем класс H2OGeneralizedLinearEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# создаем экземпляр класса H2OGeneralizedLinearEstimator
glm_model=H2OGeneralizedLinearEstimator(family='binomial', alpha=0.04, lambda_=0.008085,
                                        interactions=['GENDER', 'EDUCATION', 'MARITAL_STATUS', 'FAMILY_INCOME',
                                                      'GEN_TITLE', 'GEN_INDUSTRY', 'REGION_NM'], seed=1000000)
# обучаем модель
glm_model.train(predictors, dependent, training_frame=frame, validation_frame=newframe)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [22]:
# смотрим модель
glm_model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1520616475237_4


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.09749803516178424
RMSE: 0.3122467536449086
LogLoss: 0.33165707476479234
Null degrees of freedom: 15222
Residual degrees of freedom: 14954
Null deviance: 11112.445907685442
Residual deviance: 10097.631298288867
AIC: 10635.631298288867
AUC: 0.7242435536509765
Gini: 0.44848710730195296
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1441317492036213: 


,0,1,Error,Rate
0,10105.0,3306.0,0.2465,(3306.0/13411.0)
1,796.0,1016.0,0.4393,(796.0/1812.0)
Total,10901.0,4322.0,0.2695,(4102.0/15223.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1441317,0.3312683,217.0
max f2,0.0922374,0.4748639,291.0
max f0point5,0.2019064,0.2970030,151.0
max accuracy,0.5445162,0.8812323,8.0
max precision,0.5445162,0.6428571,8.0
max recall,0.0147295,1.0,397.0
max specificity,0.7013894,0.9999254,0.0
max absolute_mcc,0.1441317,0.2256471,217.0
max min_per_class_accuracy,0.1235371,0.6630378,244.0
max mean_per_class_accuracy,0.1214247,0.6654050,247.0


Gains/Lift Table: Avg response rate: 11,90 %



...

metric,threshold,value,idx
max f1,0.1363001,0.3163009,227.0
max f2,0.0894142,0.4634886,297.0
max f0point5,0.2102817,0.2731493,145.0
max accuracy,0.5304175,0.8828974,14.0
max precision,0.5304175,0.6,14.0
max recall,0.0131596,1.0,397.0
max specificity,0.9018547,0.9999240,0.0
max absolute_mcc,0.1213225,0.2107383,249.0
max min_per_class_accuracy,0.1221199,0.6554286,248.0
max mean_per_class_accuracy,0.1213225,0.6591079,249.0


Gains/Lift Table: Avg response rate: 11,74 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100604,0.3844337,2.8968000,2.8968000,0.34,0.34,0.0291429,0.0291429,189.6800000,189.6800000
,2,0.0200537,0.3335153,2.5731544,2.7355184,0.3020134,0.3210702,0.0257143,0.0548571,157.3154362,173.5518395
,3,0.0300469,0.3028150,2.0585235,2.5103571,0.2416107,0.2946429,0.0205714,0.0754286,105.8523490,151.0357143
,4,0.0400402,0.2828276,2.5159732,2.5117588,0.2953020,0.2948074,0.0251429,0.1005714,151.5973154,151.1758794
,5,0.0500335,0.2667719,2.5159732,2.5126005,0.2953020,0.2949062,0.0251429,0.1257143,151.5973154,151.2600536
,6,0.1,0.2167065,2.1957584,2.3542857,0.2577181,0.2763246,0.1097143,0.2354286,119.5758389,135.4285714
,7,0.1500335,0.1883347,1.7588204,2.1557085,0.2064343,0.2530174,0.088,0.3234286,75.8820375,115.5708538
,8,0.2,0.1670874,1.8984161,2.0914286,0.2228188,0.2454728,0.0948571,0.4182857,89.8416107,109.1428571
,9,0.3,0.1387530,1.4057143,1.8628571,0.1649899,0.2186452,0.1405714,0.5588571,40.5714286,86.2857143
,10,0.4,0.1176719,1.1885714,1.6942857,0.1395037,0.1988598,0.1188571,0.6777143,18.8571429,69.4285714



Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2018-03-09 21:15:48,0.000 sec,0,5556.2229538,0.3649887
,2018-03-09 21:15:48,0.297 sec,1,5093.5442509,0.3397739
,2018-03-09 21:15:48,0.443 sec,2,5055.9045906,0.3371897
,2018-03-09 21:15:49,0.638 sec,3,5054.4795498,0.3371667
,2018-03-09 21:15:49,1.089 sec,4,5048.8821845,0.3369758
,2018-03-09 21:15:49,1.337 sec,5,5048.8156491,0.3369757
